In [5]:
from urllib.request import urlopen
import json
import pandas as pd
import geopandas as gpd
import plotly.express as px

In [6]:
datapath = "../data/suitability_scores/"

# Load block group suitability scores
bg_suitability = pd.read_csv(datapath + "suitability_scores_bg.csv", dtype={"GEOID": str})
bg_suitability = bg_suitability.rename(columns={"GEOID": "fips"})

# # Load geojson file for block groups
# with urlopen("https://opendata.dc.gov/api/download/v1/items/c143846b7bf4438c954c5bb28e5d1a21/geojson?layers=2") as response:
#     bg_geojson = json.load(response)
    
bg_shapefile = gpd.read_file("/Users/jack/Downloads/2023 US Block Group Boundary File 500k/cb_2023_us_bg_500k.shp", dtype={"GEOID": str})
bg_json = bg_shapefile[['GEOID', 'geometry']].to_json()
# load a state level geojson file
with urlopen("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/refs/heads/master/data/geojson/us-states.json") as response:
    states = json.load(response)

/Users/jack/anaconda3/envs/PowerLab/lib/python3.9/site-packages/pyogrio/raw.py:196: RuntimeWarning:

driver ESRI Shapefile does not support open option DTYPE



In [8]:
bg_json = json.loads(bg_json)

In [10]:
bg_json['features'][0]

{'id': '0',
 'type': 'Feature',
 'properties': {'GEOID': '011130303001'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-85.003655, 32.478852],
    [-85.001326, 32.480041],
    [-85.001722, 32.488093],
    [-84.998848, 32.48812],
    [-84.998853, 32.489113],
    [-84.9957165356694, 32.4889767052696],
    [-84.994831, 32.486042],
    [-84.9951515506849, 32.4773871315068],
    [-85.002173, 32.477136],
    [-85.003655, 32.478852]]]}}

In [11]:
# Formula 1
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']

# sum the factors
formula_1 = bg_suitability[factors].sum(axis=1).div(len(factors))
df_formula_1 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_1})

# Formula 2
factors_plus_weights = {
    'GHI': 4,
    'Protected_Land': 4,
    'Habitat': 2,
    'Slope': 2,
    'Population_Density': 2,
    'Distance_to_Substation': 1,
    'Land_Cover': 1
}

formula_2 = bg_suitability.copy()
# Multiply each factor by its weight
for factor, weight in factors_plus_weights.items():
    formula_2[factor] = formula_2[factor] * weight
    
# sum the factors
formula_2 = formula_2[factors].sum(axis=1).div(sum(factors_plus_weights.values()))
df_formula_2 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_2})

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# custom color scale
pubu_color_scale = [
    (0.00, "#f7fcfd"),  # Very Light Blue (Unsuitable)
    (0.01, "#e0ecf4"), (0.40, "#e0ecf4"),  # Light Cyan
    (0.41, "#bfd3e6"), (0.50, "#bfd3e6"),  # Sky Blue
    (0.51, "#9ebcda"), (0.60, "#9ebcda"),  # Light Blue
    (0.61, "#8c96c6"), (0.70, "#8c96c6"),  # Moderate Blue
    (0.71, "#8c6bb1"), (0.80, "#8c6bb1"),  # Dark Teal
    (0.81, "#88419d"), (0.90, "#88419d"),  # Deep Purple-Blue
    (0.91, "#4a1486"), (1.00, "#4a1486")   # Very Dark Blue
]

# Create the choropleth map
fig = px.choropleth(
    df_formula_2,
    geojson=bg_json,
    locations='fips',  # Column in df that matches GeoJSON IDs
    featureidkey="properties.GEOID",  # Path to field in GeoJSON feature object with which to match the values passed in to locations
    color='suitability',  # Column to plot
    color_continuous_scale=pubu_color_scale,  # Custom color scale
    range_color=(0, 100),
    scope="usa",  # Focus on the United States    
)

fig.update_traces(marker_line_width=0)  # Remove bg borders

# Overlay state boundaries
fig.add_trace(go.Choropleth(
    geojson=states,  # Use the state-level GeoJSON
    locations=[feature["id"] for feature in states["features"]],  # State IDs
    z=[0] * len(states["features"]),  # Dummy data to avoid coloring
    colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],  # Transparent fill
    showscale=False,
    marker=dict(line=dict(color='black', width=1.5)),  # Black state boundaries
))

# Update layout and map appearance
fig.update_geos(
    fitbounds="locations",  # Auto-zoom based on data
    visible=False  # Hide base map
)

fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

# # add state points px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", color="statename")
# len_scatter = len(px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw").data)
# for i in range(len_scatter):
#     fig.add_trace(
#         px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw", color_discrete_sequence=px.colors.qualitative.Light24).data[i]
#     )

# Don't show the legend for the scatter points
fig.update_layout(showlegend=False)


fig.show()

In [6]:
bg_geojson.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [7]:
bg_geojson['features'][0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [8]:
bg_geojson['features'][0]['properties']

{'OBJECTID': 1,
 'TRACT': '000202',
 'BLKGRP': '1',
 'GEOID': '1500000US110010002021',
 'P0010001': 1179,
 'P0010002': 1094,
 'P0010003': 935,
 'P0010004': 43,
 'P0010005': 5,
 'P0010006': 98,
 'P0010007': 0,
 'P0010008': 13,
 'P0020002': 67,
 'P0020005': 918,
 'P0020006': 43,
 'P0020007': 1,
 'P0020008': 98,
 'P0020009': 0,
 'P0020010': 9,
 'P0030001': 1008,
 'P0030003': 800,
 'P0030004': 40,
 'P0030005': 1,
 'P0030006': 94,
 'P0030007': 0,
 'P0030008': 9,
 'P0040002': 57,
 'P0040005': 783,
 'P0040006': 40,
 'P0040007': 1,
 'P0040008': 94,
 'P0040009': 0,
 'P0040010': 5,
 'H0010001': 588,
 'H0010002': 504,
 'H0010003': 84,
 'ALAND': 192049,
 'AWATER': 0,
 'SHAPE_LENG': 2655.15168363,
 'STUSAB': 'DC',
 'SUMLEV': 150,
 'GEOCODE': '110010002021',
 'STATE': 11,
 'NAME': 'Block Group 1',
 'POP100': 1179,
 'HU100': 588,
 'P0010009': 85,
 'P0010010': 81,
 'P0010011': 2,
 'P0010012': 1,
 'P0010013': 20,
 'P0010014': 0,
 'P0010015': 52,
 'P0010016': 2,
 'P0010017': 1,
 'P0010018': 2,
 'P001001